In [1]:
from iteru import *

In [2]:
Map = Map()
Map

Map(center=[27, 31], controls=(SearchControl(found_style={'fillColor': '#3f0', 'color': '#0f0'}, marker=Marker…

In [3]:
aoi = GERD_aoi

In [4]:
SAR = SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.equals('relativeOrbitNumber_start', 50))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filter(ee.Filter.gte('system:time_start',1593228316000))\
    .filterBounds(aoi)\
    .select(['VV','VH'])

In [5]:
SAR = SAR.filter(ee.Filter.eq('system:time_start',1648178723000)).median().clip(aoi)

In [6]:
Map.add_layer_widgets(SAR,{'min':-35,'max':5,'bands':['VH']},'RAW')

In [ ]:
SAR = ee.Image(toDB(RefinedLee(toNatural(SAR)))).rename('VH_Filtered')

In [ ]:
# Map.add_layer_widgets(SAR,{'min':-35,'max':5,'bands':['VH_Filtered']},'Median Filter')

In [ ]:
histogram =  SAR.select('VH_Filtered').reduceRegion(
                                
            reducer = ee.Reducer.histogram().combine('mean', None, True).combine('variance', None, True),
            geometry = aoi, 
            scale = 10,
            bestEffort = True  
)

In [ ]:
threshold = otsu(histogram.get('VH_Filtered_histogram'))

In [ ]:
threshold.getInfo()

In [ ]:
water = SAR.select('VH_Filtered').lt(threshold).selfMask().clip(aoi).rename('water_mask')

In [ ]:
 Map.add_layer_widgets(test,{'palette': ['00FFFF', '0000FF']},'test')

In [ ]:
water_mask = water.select('water_mask').clip(GERD_aoi)

In [ ]:
feature = ee.Image(1).updateMask(water_mask).reduceToVectors(
        geometry=water_mask.geometry(),
        crs='EPSG:32636',
        scale=10,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='water_cover',
        bestEffort=True
    )

In [ ]:
# Map.add_layer_widgets(feature,{'color':'red'},'Filter Water Vector')

In [ ]:
feature = feature.map(calc_area)

In [ ]:
lake_feature = feature.sort('Area', False).first()

In [ ]:
# Map.add_ee_layer(ee.Feature(lake_feature),{'color':'black'},'Lake')

In [ ]:
# Map.add_layer_widgets(ee.Feature(lake_feature),{'color':'black'},'Lake')

In [ ]:
lake_dem = elevation_dataset.clip(lake_feature)

In [ ]:
max_ele = lake_dem.reduceRegion(
        reducer=ee.Reducer.max(),
        geometry=lake_dem.geometry(),
        scale=30,
        crs='EPSG:32636',
        maxPixels=1e11).get('DSM')

In [ ]:
water_level = max_ele.getInfo()

In [ ]:
elevations = lake_dem.reduceRegion(
        reducer=ee.Reducer.toList(),
        geometry=lake_dem.geometry(),
        maxPixels=1e11,
        scale=30,
        crs='EPSG:32636',
        bestEffort=True
    ).get('DSM')

In [ ]:
elev_pixles_num = ee.List(elevations).length()

In [ ]:
elev_sum = ee.List(elevations).reduce(ee.Reducer.sum())

In [ ]:
elevations_sum = elev_sum.getInfo()

In [ ]:
pixles_count = elev_pixles_num.getInfo()

In [ ]:
((water_level*pixles_count-elevations_sum)*900)/(1e9)